<a href="https://colab.research.google.com/github/S-Mohammad-Hashemi/AD_PPPL/blob/main/PPPL_Scenario1_CICIDS2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !nvidia-smi

In [2]:
!git clone https://github.com/S-Mohammad-Hashemi/AD_PPPL.git

Cloning into 'AD_PPPL'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 20 (delta 1), reused 9 (delta 1), pack-reused 11
Unpacking objects: 100% (20/20), done.
Checking out files: 100% (9/9), done.


In [3]:
%cd AD_PPPL/

/content/AD_PPPL


In [9]:
!bash ./extract_cicids_dataset.sh

Archive:  datasets/CICIDS2017_packet-based/tuesday.zip
   creating: datasets/CICIDS2017_packet-based/tuesday/
  inflating: datasets/CICIDS2017_packet-based/tuesday/labels.npy  
  inflating: datasets/CICIDS2017_packet-based/tuesday/part_00000.npy  
  inflating: datasets/CICIDS2017_packet-based/tuesday/part_00001.npy  
  inflating: datasets/CICIDS2017_packet-based/tuesday/part_00002.npy  
  inflating: datasets/CICIDS2017_packet-based/tuesday/part_00003.npy  
  inflating: datasets/CICIDS2017_packet-based/tuesday/part_00004.npy  
  inflating: datasets/CICIDS2017_packet-based/tuesday/part_00005.npy  
  inflating: datasets/CICIDS2017_packet-based/tuesday/part_00006.npy  
  inflating: datasets/CICIDS2017_packet-based/tuesday/part_00007.npy  
  inflating: datasets/CICIDS2017_packet-based/tuesday/part_00008.npy  
  inflating: datasets/CICIDS2017_packet-based/tuesday/part_00009.npy  
  inflating: datasets/CICIDS2017_packet-based/tuesday/part_00010.npy  
  inflating: datasets/CICIDS2017_packet-ba

In [13]:
# Path to the dataset
dataset_path='./datasets/CICIDS2017_packet-based/' 

datasets = {
    'A':'tuesday',
    'B':'wednesday',
    'C':'thursday'
    }

#### src_domain and trg_domain can be any of the above domains.
s_domain = datasets['A'] #source domain
t_domain = datasets['B'] #target domain

In [14]:
import tensorflow as tf
import numpy as np
%load_ext autoreload
%autoreload 2
import os
import sys
import time
from sklearn import metrics

In [15]:
from utils import  preproces_dataset, Solver, PacketModel, DataHandler

#Reading and preprocessing data

In [16]:
x_src,y_src,x_trg,y_trg,train_min,train_max = preproces_dataset(s_domain,t_domain,dataset_path)

##### Uncomment the next two lines to make the training procedure faster!
# x_trg = x_trg[::2]
# y_trg = y_trg[::2]

y_src_onehot = np.zeros((len(y_src),2),np.float32)
y_src_onehot[range(len(y_src)),y_src.astype(np.int32)] = 1.

y_trg_onehot = np.zeros((len(y_trg),2),np.float32)
y_trg_onehot[range(len(y_trg)),y_trg.astype(np.int32)] = 1.

print('x_src.shape:',x_src.shape,'y_src.shape:',y_src.shape,'x_trg.shape:',x_trg.shape,'y_trg.shape:',y_trg.shape)
print('Malicious ratio in the source domain:',np.sum(y_src==1)/len(y_src))
print('Malicious ratio in the target domain:',np.sum(y_trg==1)/len(y_trg))

./datasets/CICIDS2017_packet-based/tuesday/part_00000.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00001.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00002.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00003.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00004.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00005.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00006.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00007.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00008.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00009.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00010.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00011.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00012.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00013.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00014.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00015.npy
./datasets/CICIDS2017_packet-based/tuesday/part_00016.npy
./datasets/CIC

# Training a model on the src domain


In [17]:
input_size = x_src.shape[1]
net = PacketModel()
net._set_inputs(tf.TensorSpec([None,input_size]))

base_lr = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=base_lr)
grads = []
for v in net.trainable_variables:
    grads.append(np.zeros(v.shape))
optimizer.apply_gradients(zip(grads,net.trainable_variables))

tsolver = Solver(optimizer,net,base_lr)
train_loss = tf.keras.metrics.Mean(name='train_loss')

In [20]:
def train_step_eager(x, y, optimizer,net):
    with tf.GradientTape() as tape:
        y_pred = net(x, training=True)
        mse_loss = tf.reduce_sum((y - y_pred)**2,axis=1)
        mse_loss = tf.reduce_mean(mse_loss)
    gradients = tape.gradient(mse_loss, net.trainable_variables)
    optimizer.apply_gradients(zip(gradients, net.trainable_variables))
    train_loss(mse_loss)

train_step = tf.function(train_step_eager)

In [21]:
def train_model_on_src(nb_iters,dhandler,tsolver):
    st = time.time()
    train_loss.reset_states()
    for i in range(nb_iters):
        x_batch,y_batch = dhandler.next_batch()

        if i%5==0:
            tsolver.iters+=1
            tsolver.update_lr()

        train_step(x_batch,y_batch,tsolver.optimizer,tsolver.net)
        if i % 50 == 49 or i == nb_iters - 1:
            remained_iters = nb_iters - i
            passed_time = time.time() - st
            ETA = int(passed_time * remained_iters / i)
            ETA_min, ETA_sec = ETA // 60, ETA % 60
            mean_loss = train_loss.result().numpy()
            print ('\r' + \
                  ' iter: ' + str(i + 1) + '/' + str(nb_iters) + \
                  ' ETA: ' + str(ETA_min) + ':' + "{0:02d}".format(ETA_sec) + \
                  ' loss: ' + "{0:0.4f}".format(mean_loss),end=" ")
            sys.stdout.flush()
    print(' ')



In [22]:
nb_epochs = 6
batch_size = 256
total_batch = len(x_src)//batch_size
if len(x_src) % batch_size!=0:
    total_batch+=1
RANDOM_SEED = 2022
rng = np.random.RandomState(RANDOM_SEED)


for i in range(nb_epochs):
    pos_inds = y_src==1
    x_src_pos = x_src[pos_inds]
    y_src_pos = y_src_onehot[pos_inds]
    x_src_neg = x_src[~pos_inds]
    y_src_neg = y_src_onehot[~pos_inds]
    p = np.random.permutation(len(x_src_neg))
    x_src_neg = x_src_neg[p]
    y_src_neg = y_src_neg[p]
    pos_len = len(x_src_pos)

    src_dhandler = DataHandler(np.concatenate((x_src_pos,x_src_neg[:pos_len]))
                                        ,np.concatenate((y_src_pos,y_src_neg[:pos_len])),None,batch_size=256,shuffle=True)

    train_model_on_src(total_batch,src_dhandler,tsolver)

 iter: 2241/2241 ETA: 0:00 loss: 0.0367  
 iter: 2241/2241 ETA: 0:00 loss: 0.0142  
 iter: 2241/2241 ETA: 0:00 loss: 0.0118  
 iter: 2241/2241 ETA: 0:00 loss: 0.0087  
 iter: 2241/2241 ETA: 0:00 loss: 0.0088  
 iter: 2241/2241 ETA: 0:00 loss: 0.0076  


In [23]:
def test_step_eager(x,net):
    return net(x,training=False)

test_step = tf.function(test_step_eager)

In [24]:
def test(x_ds,y_ds,net,ret=False):
    all_y_pred = np.zeros_like(y_ds)
    all_scores = np.zeros((len(y_ds),2))
    for i in range(0,len(x_ds),batch_size):
        x_batch = x_ds[i:i+batch_size]
        y_batch = y_ds[i:i+batch_size]
        y_pred = test_step(x_batch,net)
        y_pred = y_pred.numpy()
        all_scores[i:i+batch_size] = y_pred
        y_pred = y_pred.argmax(axis=1)
        all_y_pred[i:i+batch_size] = y_pred
    if ret:
        return all_y_pred,all_scores
    print('accuracy:',np.sum(all_y_pred==y_ds)/len(y_ds))


In [25]:
def calc_f1_score(x_ds, y_ds, net):
    all_y_pred,_ = test(x_ds,y_ds,net,ret=True)
    f1_score = metrics.f1_score(y_true=y_ds,y_pred=all_y_pred)
    return f1_score

In [26]:
only_src_f1_score = calc_f1_score(x_trg,y_trg, net)
print('F1 score on the trg domain:',only_src_f1_score)

F1 score on the trg domain: 0.06420724857313037


# Domain Adaptation with PPPL


In [27]:
def adjust_pseudo_labels(trg_probs_np,trg_cp):
    n_classes = 2
    pseudo_labels = trg_probs_np.argmax(axis=1)
    current_cp = np.zeros(n_classes)
    for c in range(n_classes):
        current_cp[c] = np.sum(pseudo_labels==c)/len(trg_probs_np)

    diff_class_rates =  current_cp - trg_cp
    for i in range(len(diff_class_rates)):
        if diff_class_rates[i]<=0:
            continue
        predicted_as_c = pseudo_labels==i
        current_class = i
        current_diff = diff_class_rates[i]
        current_num = np.round(current_diff*len(trg_probs_np)).astype(np.int32)

        current_probs = trg_probs_np[pseudo_labels==current_class]
        current_probs_sorted = np.sort(current_probs,axis=1)
        current_certainty_scores = current_probs_sorted[:,-1] - current_probs_sorted[:,-2]
        
        current_certainty_scores_sorted_inds = np.argsort(current_certainty_scores)
        y_val = np.ones(len(current_certainty_scores))*current_class
        for j in range(current_num):
            y_val[j]=1-current_class ###change pseudo-label to the opposite class!
#             y_val[j]=-1
        temp_pl = np.zeros(len(current_certainty_scores))
        temp_pl[current_certainty_scores_sorted_inds] = y_val
        pseudo_labels[predicted_as_c] = temp_pl
    
    return pseudo_labels

In [28]:
def train_step_DA_eager(x, y, w, optimizer,net):
    with tf.GradientTape() as tape:
        y_pred = net(x, training=True)
        mse_loss = tf.reduce_sum((y - y_pred)**2,axis=1)*w
        mse_loss = tf.reduce_mean(mse_loss)
    gradients = tape.gradient(mse_loss, net.trainable_variables)
    optimizer.apply_gradients(zip(gradients, net.trainable_variables))
    train_loss(mse_loss)

train_step_DA = tf.function(train_step_DA_eager)

In [29]:
def train_model_with_weights(nb_epochs,dhandler,tsolver):
    total_batch = dhandler.len // dhandler.batch_size
    if dhandler.len % dhandler.batch_size != 0:
        total_batch += 1
    st = time.time()
    for ep in range(nb_epochs):
        train_loss.reset_states()
        for i in range(total_batch):
            x_batch,y_batch_t,w_batch = dhandler.next_batch()
            y_batch = np.zeros((dhandler.batch_size,2),dtype=np.float32)
            y_batch[range(dhandler.batch_size),y_batch_t] = 1
            train_step_DA(x_batch,y_batch,w_batch,tsolver.optimizer,tsolver.net)

        passed_time = time.time() - st
        remained_epochs = nb_epochs - ep
        ETA = int(passed_time * remained_epochs)
        ETA_min, ETA_sec = ETA // 60, ETA % 60
        print ('\r' + 'epoch: ' + str(ep + 1) + '/' + str(nb_epochs) + \
                      ' ETA: ' + str(ETA_min) + ':' + "{0:02d}".format(ETA_sec) + \
                      ' loss: ' + "{0:0.4f}".format(train_loss.result().numpy()),end=" ")
        sys.stdout.flush()
    print(' ')

In [30]:
def DA(tsolver,trg_data,trg_gts,src_data,src_gts,trg_cp,W = 0.):
    begin_time = time.time()
    my_coef = 0.05
    weights_src = np.ones(len(src_gts))
    trg_gts_unreal = np.zeros(len(trg_data))


    for nnn in range(0,90,2):
        print ('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ i:',nnn//2 + 1,
               'Elapsed Time(m): {0:0.2f}'.format((time.time()-begin_time)/60))
        
        #### Get scores and pseudo labels of the target domain
        trg_pseudo_labels, trg_scores_np  = test(trg_data,trg_gts_unreal,tsolver.net,ret=True)

        if nnn<60:
            # Update target class proportions for scenario 3
            for i in range(2):
                temp = np.sum(trg_pseudo_labels==i)/len(trg_pseudo_labels)
                trg_cp[i] = (1-W)*trg_cp[i] + W*temp

        trg_pseudo_labels_adjusted = adjust_pseudo_labels(np.copy(trg_scores_np),trg_cp)
        if nnn<60:
            ### for the first 30 iterations instead of exclusion of samples 
            ### that their CP is larger than expected CP we change their pseudo-label
            ### to the oposite class and keep them into the training set!
            trg_pseudo_labels = trg_pseudo_labels_adjusted

        ### Calculate the certainty scores for target samples
        trg_scores_np_sorted = np.sort(trg_scores_np,axis=1)
        certainty_scores = trg_scores_np_sorted[:,-1] - trg_scores_np_sorted[:,-2]
        

        ### Calculate weight for the target samples
        weights_trg = np.zeros(len(certainty_scores))
        for c in range(n_classes):
            predicted_as_c = trg_pseudo_labels==c
            size_c = np.sum(predicted_as_c)
            if size_c>1:
                left_size = int(np.ceil(((nnn+1)*0.01+0.1)*size_c))
                x_val_left = 1+(10/2 - 1)/left_size*(np.arange(left_size))
                right_size = size_c - left_size
                x_val_right = 10000*(np.arange(1,right_size+1))
                x_val = np.concatenate((x_val_left,x_val_right))
                y_val = np.power(x_val,-1)
                y_val = y_val[::-1]

                cs_c = certainty_scores[predicted_as_c]
                cs_c_sorted_inds = np.argsort(cs_c)
                weights_trg2 = np.zeros(len(cs_c))
                weights_trg2[cs_c_sorted_inds] = y_val
                weights_trg[predicted_as_c] = weights_trg2
                
        ### Exclude
        coef = (trg_pseudo_labels==trg_pseudo_labels_adjusted)*1
        weights_trg*=coef
        inclusion_condition = weights_trg>=0.001
        trg_samples = trg_data[inclusion_condition]
        trg_pseudo_labels = trg_pseudo_labels[inclusion_condition].astype(np.int32)
        weights_trg = weights_trg[inclusion_condition]


        #### Randomly select some samples from the source domain
        p = np.random.permutation(len(src_data))
        p = p[:len(trg_data)]
        x_temp = src_data[p]
        y_temp = src_gts[p]
        w_temp = weights_src[:len(trg_data)]

        #### Train Model
        m1 = np.concatenate((x_temp,trg_samples))
        m2 = np.concatenate((y_temp,trg_pseudo_labels)).astype(np.int32)
        m3 = np.concatenate((w_temp,weights_trg)).astype(np.float32)
        
                                    
        ### Balancing the positive and negative samples
        pos_inds = m2==1
        x_train_pos = m1[pos_inds]
        y_train_pos = m2[pos_inds]
        w_train_pos = m3[pos_inds]
        x_train_neg = m1[~pos_inds]
        y_train_neg = m2[~pos_inds]
        w_train_neg = m3[~pos_inds]
        p = np.random.permutation(len(x_train_neg))
        x_train_neg = x_train_neg[p]
        y_train_neg = y_train_neg[p]
        w_train_neg = w_train_neg[p]
        pos_len = len(x_train_pos)

        DA_dhandler = DataHandler(np.concatenate((x_train_pos,x_train_neg[:pos_len]))
                                ,np.concatenate((y_train_pos,y_train_neg[:pos_len]))
                                ,np.concatenate((w_train_pos,w_train_neg[:pos_len])),batch_size=256,shuffle=True)            

        ep = 1
        train_model_with_weights(ep,DA_dhandler,tsolver)

        if nnn%8==0:
            current_f1 = calc_f1_score(trg_data,trg_gts, tsolver.net)
            print('Current F1 score on the trg domain:',current_f1)

In [31]:
base_lr = 0.0001
optimizer = tf.keras.optimizers.Adam(lr=base_lr)
grads = []
for v in net.trainable_variables:
    grads.append(np.zeros(v.shape,dtype=np.float32))
optimizer.apply_gradients(zip(grads,net.trainable_variables))

tsolver = Solver(optimizer,net,base_lr)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [32]:
trg_gts = y_trg
trg_data = x_trg
src_gts = y_src
src_data = x_src
t_labels = np.array(trg_gts)
s_labels = np.array(src_gts)

n_classes = 2

trg_gt_class_percentage = np.zeros(n_classes)
for i in range(n_classes):
    #Perfect knowledge about target domain class proportions
    trg_gt_class_percentage[i] = np.sum(t_labels==i)/len(t_labels)


In [33]:
DA(tsolver,trg_data,trg_gts,src_data,src_gts,trg_gt_class_percentage)

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ i: 1 Elapsed Time(m): 0.00
epoch: 1/1 ETA: 0:00 loss: 0.1433  
Current F1 score on the trg domain: 0.31527647378564017
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ i: 2 Elapsed Time(m): 0.14
epoch: 1/1 ETA: 0:00 loss: 0.0419  
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ i: 3 Elapsed Time(m): 0.23
epoch: 1/1 ETA: 0:00 loss: 0.0162  
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ i: 4 Elapsed Time(m): 0.32
epoch: 1/1 ETA: 0:00 loss: 0.0142  
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ i: 5 Elapsed Time(m): 0.40
epoch: 1/1 ETA: 0:00 loss: 0.0132  
Current F1 score on the trg domain: 0.707754563622929
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ i: 6 Elapsed Time(m): 0.54
epoch: 1/1 ETA: 0:00 loss: 0.0117  
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [34]:
current_f1 = calc_f1_score(trg_data,trg_gts,tsolver.net)
print('Current F1 score on the trg domain:',current_f1)

Current F1 score on the trg domain: 0.9697620094647533
